<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_45_02_Pull_Print_StandAlone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---



[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

Copyright (c) 2022, Prithwis Mukerjee All rights reserved.

This source code is licensed under the GNU GPL v3.0 -style license found in the LICENSE file in the root directory of this source tree.

For those who may not have access to a live MongoDB instance, we have set up a mechanism where the JSON file that would have been loaded into a hosted MongoDB instance, will be loaded into a locally (that is in the VM) installed MongoDB instance and the queries can be run in a manner that is identical to a live, hosted MongoDB instance

#SetUp Environment

##Install Software

In [1]:
!pip -qq install python-docx                #https://python-docx.readthedocs.io/en/latest/
!apt install mongodb > /dev/null            # install local MongoDB server           
!service mongodb start                      # start Local MongoDB server instance


     |████████████████████████████████| 5.6 MB 4.0 MB/s 


 * Starting database mongodb
   ...done.


##Install, import prerequisites

In [2]:
#Utility functions 
from pymongo import MongoClient
import pandas as pd
import dateutil
import json
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
from datetime import timedelta
import pytz

#import matplotlib.pyplot as plt
#import math


In [3]:
#Load p21 modules from github
!wget -O p21.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21.py                  # all global variables
!wget -O p21utils.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utils.py        # large number of utility functions
!wget -O p21utilsR.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utilsR.py 
!wget -O p21YogInfo.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21YogInfo.py
# ---------------------------

!wget -O Saraswati.png -q https://github.com/prithwis/parashar21/raw/main/images/Saraswati02.png                 
!wget -O p21logo.png -q https://github.com/prithwis/parashar21/raw/main/images/p21logo-002-1.png                  

In [4]:
#!cat p21utils.py
import p21
import p21utils
import p21utilsR
import p21YogInfo
# ------------------------------------------------
# required for multiple imports of the same module
# ------------------------------------------------
import importlib
importlib.reload(p21)
importlib.reload(p21utils)
importlib.reload(p21utilsR)
importlib.reload(p21YogInfo)

p21utils imported
p21utilsR imported
p21utils imported
p21utilsR imported


<module 'p21YogInfo' from '/content/p21YogInfo.py'>

##Define MongoDB database in local VM

In [5]:
mongoURI = "mongodb://localhost:27017/"                               # for remote server
client = MongoClient(mongoURI)

db = client.parashar21Local
#pd.DataFrame(list(db.list_collections()))

##Download Horoscope data in JSON and load into MongoDB

In [6]:
# Retrieve JSON file from Github and load in local Mongo
#!wget -O peopleData.json -q https://github.com/prithwis/parashar21/raw/main/data/peopleDataTest5.json

#!gdown -O peopleData.json https://drive.google.com/uc?id=1_VBHE7Hyue8t3hdBdLU6lgpnaWhhRVd4  # 39662 documents OLD file, NOT TO BE USED
!gdown  -O peopleData.json https://drive.google.com/uc?id=1z4JFd-zCaS71dICKIYx46kQGbXPAHq5C  # 39662 documents
!mongoimport --db parashar21Local --collection khona21Test --drop --file peopleData.json

Downloading...
From: https://drive.google.com/uc?id=1z4JFd-zCaS71dICKIYx46kQGbXPAHq5C
To: /content/peopleData.json
100% 134M/134M [00:01<00:00, 106MB/s]
2022-09-19T09:05:17.975+0000	connected to: localhost
2022-09-19T09:05:17.976+0000	dropping: parashar21Local.khona21Test
2022-09-19T09:05:20.976+0000	[#####...................] parashar21Local.khona21Test	28.2MB/128MB (22.0%)
2022-09-19T09:05:23.981+0000	[##########..............] parashar21Local.khona21Test	56.8MB/128MB (44.3%)
2022-09-19T09:05:26.983+0000	[################........] parashar21Local.khona21Test	85.5MB/128MB (66.8%)
2022-09-19T09:05:29.974+0000	[#####################...] parashar21Local.khona21Test	114MB/128MB (89.2%)
2022-09-19T09:05:31.431+0000	[########################] parashar21Local.khona21Test	128MB/128MB (100.0%)
2022-09-19T09:05:31.431+0000	imported 39662 documents


In [8]:
p21.kollection = db.khona21Test                   # Set the collection to work with
#kollection = db.khona21                   # Set the collection to work with
totalCharts = p21.kollection.count_documents({})
dbSize = db.command( {"dbStats": 1, "scale": 1024 } )['dataSize']
print(totalCharts, ' documents,',round(dbSize,2), " KB data")

39662  documents, 142321.85  KB data


#Report Generation Function
Please note that the generated report MS Office doc format **will be stored in the Colab VM drive** and needs to be downloaded to local machine for viewing

In [9]:
def genSampleReport(ChartStyle,Sample = True):
    
    if Sample:
        df4Report = df.sample(5)
    else:
        df4Report = df
    
    selectedCharts = len(df)
    printCharts = len(df4Report)
    #printStatus = str(printCharts)+' charts of total '+str(totalCharts)+' printed.'
    printStatus = str(selectedCharts)+' Charts selected from '+str(totalCharts)+' in database; '+str(printCharts)+' charts printed.'

    now = datetime.now(pytz.timezone('Asia/Kolkata'))
    fileName = 'p21Report_'+ChartStyle+'_'+now.strftime("%H%M%S")+'.doc'
    curQueryStr = json.dumps(selCond)
    # ---------------------------------------------------------
    p21utilsR.R01_CreateReportDoc(curQueryStr, printStatus)                      # Creates MS Word Doc called p21.document
    # ----------------------------------------------------------
    for chart in range(len(df4Report)):
        chartData =df4Report.iloc[chart]
        #R51_processChart(chartData)
        p21utilsR.R511_parseChartData(chartData)              # Break up database data into useful pieces
                                            # Creates two primary dicts, GLon, GRet
        for ctype in ['Rashi','Navamsa']:
        #for ctype in ['Rashi']:
            p21.ChartType = ctype
            p21utils.R11_LocateGrahaInRashi()   # Converts Graha Long to Rashi positions
                                                # Rashi Num as well as Rashi name
            if ChartStyle == 'Bengal':
                p21utilsR.R12B_drawChart_Bengal()    # Draw Chart in Bengal style
            if ChartStyle == 'South':
                p21utilsR.R12B_drawChart_South()    # Draw Chart in South style
            if ChartStyle == 'North':
                p21utilsR.R12B_drawChart_North()    # Draw Chart in North style
            #
            
                                                
        #print(p21.GrahaBhava)
        #print(p21.LordBhav)
        p21utilsR.R30_LocateYogs()
        p21utilsR.R512_FormatPage()
        #print('----')
    # ------------------------------------
    p21.document.save(fileName)
    


#Retrieve Charts 
Based on selective queries <br>
All output messages suppressed <br>
Reports will be available as DOC files in the VM drive <br>
Please specify style of report, 'Bengal', 'North' or 'South' while calling the report generation routine <br>
genSampleReport('Bengal',Sample = False), specifying Sample='False' will stop sampling and print reports for all charts retrieved. Otherwise, by default, only 5 charts, chosen at random will be printed in the DOC file


In [ ]:
'''
c = kollection.find().limit(5)
for crnt in c:
    print(crnt)
'''

'\nc = kollection.find().limit(5)\nfor crnt in c:\n    print(crnt)\n'

In [ ]:
%%capture
selCond = {}                      # All Charts 
df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
genSampleReport('Bengal',Sample = False)

In [12]:
%%capture
selCond = {'GAspectedBy2.La': {'$in': ['Sa']}}                      # La aspected by Sa
df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
genSampleReport('South')

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}}       # La aspected by Sa          

                    ]
}

df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
genSampleReport('North')

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa       
                        {'GConjunctsG2.Su' : {'$in': ['Me']}}       # Su conjunct Me 

                    ]
}

df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
genSampleReport('Bengal')

In [11]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa       
                        {'GConjunctsG2.Su' : {'$in': ['Me']}},      # Su conjunct Me 
                        {'GrahaBhava.Mo' : {'$eq': 1}}              # Mo in First House

                    ]
}

df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
genSampleReport('Bengal')

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa       
                        {'GConjunctsG2.Su' : {'$in': ['Me']}},      # Su conjunct Me 
                        {'GrahaBhava.Mo' : {'$eq': 1}} ,            # Mo in First House
                        {'LordBhav.4' : {'$eq': 5}}                 # 4th Lord in 5th House

                    ]
}

df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
genSampleReport('Bengal',Sample = False)

In [ ]:
!rm *.doc

In [ ]:
print("Last executed ..", datetime.now(pytz.timezone('Asia/Kolkata')))

Last executed .. 2022-09-18 10:56:55.607921+05:30
